In [ ]:
import numpy as np
import tensorflow as tf
import os

class Model:

    saved_speed_model = 'speed_model/'
    saved_angle_model = 'angle_model/'
    saved_signal_model = 'signal_model/'
    
    def __init__(self):
        self.speed_model = tf.keras.models.load_model(os.path.join(os.path.dirname(os.path.abspath(__file__)), self.saved_speed_model))
        self.angle_model = tf.keras.models.load_model(os.path.join(os.path.dirname(os.path.abspath(__file__)), self.saved_angle_model))
        self.signal_model = tf.keras.models.load_model(os.path.join(os.path.dirname(os.path.abspath(__file__)), self.saved_signal_model))

    def preprocess(self, image):
        img = tf.image.convert_image_dtype(image, tf.float32)
        img = tf.image.resize(img,(224, 224))
        img = tf.reshape(img,(1,224,224,3))
        return img

    def predict(self, image):
        image = self.preprocess(image)
        angle_value = self.angle_model.predict(image)[0][0]
        speed_value = self.speed_model.predict(image)[0][0]
        signal_value = self.signal_model.predict(image)[0]
        
        if max(signal_value) != signal_value[0]:
            if abs(signal_value[1]-signal_value[2]) > 0.1:
                if signal_value[1] > signal_value[2]:
                    speed = 1
                else:
                    speed = 0
            else:
                speed = speed_value
        else:
            speed = speed_value
        if abs((1 - speed)) < abs((0-speed)):
            speed = 30
        else:
            speed = 0
        angle = angle_value * 80 + 50
        angle = 5 * round(angle/5)
        if angle < 50:
            angle = 50
        if angle > 130:
            angle = 130

        print('angle:', angle,'speed:', speed)
        
        return angle, speed



In [1]:
import numpy as np
import tensorflow as tf
import os

In [3]:
import tensorflow as tf
speed_model = tf.keras.models.load_model("speed_model")

2022-05-11 15:08:14.405561: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-11 15:08:14.405703: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Max


In [6]:

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model("angle_model") # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('angle_model.tflite', 'wb') as f:
    f.write(tflite_model)

2022-05-11 15:10:18.204800: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2022-05-11 15:10:18.204816: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2022-05-11 15:10:18.204818: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:372] Ignored change_concat_input_ranges.
2022-05-11 15:10:18.204892: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: angle_model
2022-05-11 15:10:18.224611: I tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2022-05-11 15:10:18.224631: I tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: angle_model
2022-05-11 15:10:18.292716: I tensorflow/cc/saved_model/loader.cc:210] Restoring SavedModel bundle.
2022-05-11 15:10:18.575230: I tensorflow/cc/saved_model/loader.cc:194] Running initialization op on SavedModel bundle at path: angle_model
2022-05-11 15:10: